In [1]:
import numpy as np
import pandas as pd
import timeit

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

import pickle as pkl
from joblib import dump, load



# Loading data

In [2]:
# load training dataset
with open('datasets/train.dataset.6mer.npy', 'rb') as open_file:
    df = np.load(open_file)
df = pd.DataFrame(df)

In [3]:
six_mers = pd.read_table('datasets/6mer_columns.txt', header=None)
df.columns = six_mers[:2080]
df.columns = [col[0] for col in df.columns]

In [4]:
labels = pd.read_csv('datasets/train_labels.csv')
df['genome_label'] = labels
df.head()

,AAAAAA,AAAAAT,AAAAAG,AAAAAC,AAAATA,AAAATT,AAAATG,AAAATC,AAAAGA,AAAAGT,...,CCATGG,CCAGGG,CCACGG,CCTAGG,CCCAGG,CCGAGG,CCCCGG,CCGCGG,CCCGGG,genome_label
0,0.002470,0.004528,0.003292,0.000823,0.003704,0.002880,0.001646,0.001646,0.001646,0.001646,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
1,0.001818,0.002857,0.002077,0.001558,0.003635,0.002338,0.002338,0.001039,0.001818,0.001039,...,0.000519,0.0,0.0,0.000519,0.000260,0.0,0.0,0.0,0.0,staphylococcus_aureus
2,0.003702,0.003084,0.001234,0.001851,0.002468,0.003084,0.003084,0.001851,0.001234,0.000617,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
3,0.001102,0.002756,0.003584,0.001378,0.003307,0.002481,0.002481,0.001102,0.001654,0.001378,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,staphylococcus_aureus
4,0.004318,0.003534,0.002748,0.001701,0.003534,0.002356,0.002224,0.002617,0.003271,0.001440,...,0.000000,0.0,0.0,0.000262,0.000131,0.0,0.0,0.0,0.0,staphylococcus_aureus


In [5]:
df.shape

(505536, 2081)

# feature selection

In [6]:
# data cleaning
sum_row = df.iloc[:,:-1].sum(axis=1)
print(sum_row.describe())
print((sum_row < 0.9).sum())

# to remove samples with low kmer count
print(df.shape)
df = df.loc[sum_row >= 0.9,:]
print(df.shape)
y_index = labels[sum_row >= 0.9]

count    505536.000000
mean               NaN
std           0.000000
min           0.000000
25%           0.975098
50%           1.010742
75%           1.039062
max           1.311523
dtype: float64
22011
(505536, 2081)
(483525, 2081)


In [7]:
sum_all = df.groupby('genome_label').sum()
s_describe = sum_all.T.describe()
select = sum_all.T > s_describe.loc['75%',:]
select_sum = select.sum(axis=1)
select_sum

AAAAAA    24
AAAAAT    26
AAAAAG    26
AAAAAC    26
AAAATA    24
          ..
CCCAGG     7
CCGAGG     7
CCCCGG     6
CCGCGG     9
CCCGGG     6
Length: 2080, dtype: int64

In [8]:
selected = select_sum>15

In [9]:
df = df.iloc[:,:-1].loc[:,selected]


In [10]:
corr_matrix = df.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # remove duplicate upper triangle correlation values
corr_matrix = corr_matrix.abs() # create positive correlation matrix
tri_df = corr_matrix.mask(mask) # create and apply upper triangle mask

In [11]:
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.7)]
df.drop(columns=to_drop, inplace=True)
len(to_drop)

53

# sampling

In [12]:
def extract_nc(filename):
    nc = []
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('>'):
                header = line.strip().lstrip('>')
                parts = header.split()
                #print(parts)
                if len(parts) >= 1:
                    x = parts[1]
                    x = x.split(',')[0]
                    nc.append(x.split('.')[0])
    return nc

In [13]:
filename = 'datasets/train.dataset.raw.reads.fna'
species = extract_nc(filename)

In [14]:
# labelling for the rfseq numbers

sample_label = labels.copy()
sample_label['rfseq'] = species
print(sample_label.shape)
# sample_label.value_counts().to_csv('test.csv')
sample_label

(505536, 2)


,genome_name,rfseq
0,staphylococcus_aureus,NC_007795
1,staphylococcus_aureus,NC_007795
2,staphylococcus_aureus,NC_007795
3,staphylococcus_aureus,NC_007795
4,staphylococcus_aureus,NC_007795
...,...,...
505531,decoy,NC_000004
505532,decoy,NC_000017
505533,decoy,NC_000011
505534,decoy,NT_167249


In [15]:
def sampling(x, frac=None):
    """" subsampling certain fraction, unless when the species is only 1 """
    current = x.shape[0]
    if current==1:
        return x
    x = x.sample(frac=frac, random_state=4)
    return x

In [16]:
sample_label = sample_label.loc[sum_row >= 0.9,:]
x = sample_label[sample_label['genome_name']=='decoy'].groupby('rfseq').apply(lambda x: sampling(x,0.2)).index.get_level_values(1) # index for selected decoys
x2 =sample_label[sample_label['genome_name']!='decoy'].index # index for all organisms
x = x.append(x2) # index for all selected samples
x

Index([421679, 285281, 481246, 263189, 105081, 355473, 317703, 278399, 321451,
       470007,
       ...
        75460,  75461,  75462,  75463,  75464,  75465,  75466,  75467,  75468,
        75469],
      dtype='int64', length=144030)

In [17]:
df_train = df.loc[x, :]
df_train['genome_label'] = y_index
df_train.shape

(144030, 290)

In [65]:
df_train['genome_label'].value_counts()

genome_label
decoy                              84893
burkholderia_pseudomallei           3787
pseudomonas_aeruginosa              3342
klebsiella_michiganensis            3166
mycobacterium_ulcerans              2999
klebsiella_pneumoniae               2840
serratia_liquefaciens               2832
citrobacter_freundii                2715
salmonella_enterica_typhimurium     2594
salmonella_enterica_paratyphi       2578
yersinia_enterocolitica             2404
stenotrophomonas_maltophilia        2388
mycobacterium_tuberculosis          2354
clostridioides_difficile            2248
acinetobacter_baumannii             2124
legionella_pneumophila              1801
vibrio_parahaemolyticus             1738
listeria_monocytogenes              1576
vibrio_cholerae                     1561
staphylococcus_aureus               1489
staphylococcus_pseudintermedius     1376
corynebacterium_ulcerans            1306
corynebacterium_diphtheriae         1274
neisseria_meningitidis              1194
str

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(df_train.iloc[:,:-1],y_index,random_state=4,test_size=0.2, stratify=y_index)


# dimensional reduction

In [18]:
def create_pca(data, seed=4220, save=None):
    """ Returns PCA & transformed data """
    from sklearn.decomposition import PCA
    import pickle as pkl
    print("shape of data: ", data.shape)

    new_pca = PCA(n_components=None, random_state=seed).fit(data)

    lim = 0.9
    ACC_VAR = 0
    for i, var in enumerate(new_pca.explained_variance_ratio_):
        ACC_VAR+=var
        # print(var)
        if ACC_VAR > lim: 
            print(f"{i+1} components explained {lim}S of total var")
            break
    
    new_pca = PCA(n_components=i+1, random_state=seed) 
    data = new_pca.fit_transform(data)
    print(new_pca)
    print("shape of new data: ", data.shape)

    if save!=None:
        with open(save, 'wb') as pickle_file:
            pkl.dump(new_pca, pickle_file)
        print("model saved")

    return [new_pca, data]

In [19]:
new_pca, pca_data = create_pca(df_train.iloc[:,:-1], save='models/pca_ver2.pkl')


shape of data:  (144030, 289)
156 components explained 0.9S of total var
PCA(n_components=156, random_state=4220)
shape of new data:  (144030, 156)
model saved


In [ ]:
svd = TruncatedSVD(n_components=500, random_state=4220)
svd_data = svd.fit_transform(df_train.iloc[:, :-31]) 

with open('svd_n500.pkl', 'wb') as pickle_file:
    pkl.dump(svd, pickle_file)

# model training

In [23]:
def create_svm(x_data, y_label, save=None):
    """ training support vector machine """
    from sklearn.svm import SVC
    from joblib import dump
    import timeit

    print("training")
    starting_time = timeit.default_timer()

    clf = SVC(kernel='rbf', probability=True)
    clf.fit(x_data , y_label)
    print(clf)

    print("Time taken :", timeit.default_timer() - starting_time)

    if save != None:   
        dump(clf, save) 
        print("model saved")
    
    return clf

In [24]:
pca_clf = create_svm(pca_data , df_train['genome_label'].values, save='models/svm_ver2.joblib')

training


# testing

In [ ]:
from function import precision_per_patient
from joblib import dump, load
#load trained model
clf = load('svm_ver2.joblib')

with open('pca_ver2.pkl', 'rb') as pickle_file: # PCA embeddings trained on full data
    preprocess=pkl.load(pickle_file) 

In [ ]:
#prediction for all patients
def run_test(threshold=0.99, model=clf, preprocess=None):
    all_precision = []
    for patient_id in range(1,11):
        print('predicting for patient {}'.format(patient_id))
        
        starting_time = timeit.default_timer()
        with open('datasets/validation/patient{}.6mer.npy'.format(patient_id), 'rb') as read_file:
            df_test = np.load(read_file)
            df_test = pd.DataFrame(df_test)
            
        df_test.columns = six_mers[:2080]
        df_test.columns = [col[0] for col in df.columns]    
        df_test = df_test.loc[:, df_train.columns]
        df_test = preprocess.transform(df_test)
        
        y_predprob = model.predict_proba(df_test)
        
        #we get only predictions larger than the threshold and if there is more than one, we take the argmax again
        final_predictions = le.inverse_transform(np.unique([np.argmax(item) for item in y_predprob  if len(np.where(item>= threshold)[0]) >=1]
                                                    ))
        #my pathogens dectected, decoy will be ignored
        final_predictions = [item for item in final_predictions if item !='decoy']
        
        precision = precision_per_patient(patient_id, final_predictions)
        print('precision: {}'.format(precision))
        all_precision.append(precision)
        print("Time taken :", timeit.default_timer() - starting_time)
    # performance per patient and its final average
    print([f'patient {c}: {item}' for c, item in enumerate(all_precision, start=1)])
    print(f'avg: {np.mean(all_precision)}')
    return round(np.mean(all_precision), 5)

run_test(model=clf, preprocess=preprocess, threshold=0.99)